## 编程实践部分

#### 1. 设计你自己的句子生成器

如何生成句子是一个很经典的问题，从1940s开始，图灵提出机器智能的时候，就使用的是人类能不能流畅和计算机进行对话。和计算机对话的一个前提是，计算机能够生成语言。

计算机如何能生成语言是一个经典但是又很复杂的问题。 我们课程上为大家介绍的是一种基于规则（Rule Based）的生成方法。该方法虽然提出的时间早，但是现在依然在很多地方能够大显身手。值得说明的是，现在很多很实用的算法，都是很久之前提出的，例如，二分查找提出与1940s, Dijstra算法提出于1960s 等等。

在著名的电视剧，电影《西部世界》中，这些机器人们语言生成的方法就是使用的SyntaxTree生成语言的方法。

> 
>

![WstWorld](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1569578233461&di=4adfa7597fb380e7cc0e67190bbd7605&imgtype=0&src=http%3A%2F%2Fs1.sinaimg.cn%2Flarge%2F006eYYfyzy76cmpG3Yb1f)

> 
>

在这一部分，需要各位同学首先定义自己的语言。 大家可以先想一个应用场景，然后在这个场景下，定义语法。例如：

在西部世界里，一个”人类“的语言可以定义为：
``` 
human = """
human = 自己 寻找 活动
自己 = 我 | 俺 | 我们 
寻找 = 看看 | 找找 | 想找点
活动 = 乐子 | 玩的
"""
```

一个“接待员”的语言可以定义为
```
host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = 耍一耍 | 玩一玩
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？"""

```




请定义你自己的语法: 

第一个语法：

In [10]:
teacher = '''
teacher = 学科 职位 行为 对象
学科 = 数学 | 英语 | 体育
职位 = 教授 | 助教 | 教师
行为 = 教 | 听 | 讲
对象 = 小学生 | 中学生 | 大学生
'''

第二个语法：

In [11]:
student = '''
student = 人称 喜好 动作 课程
人称 = 你 | 我 | 他
喜好 = 喜欢 | 愿意 | 不想
动作 = 听 | 学 | 练
课程 = 数学 | 语文 | 物理
'''

TODO: 然后，使用自己之前定义的generate函数，使用此函数生成句子。

In [12]:
import random

In [25]:
def generate(grammar_rule,target):
    if target in grammar_rule:
        #print(target)
        candidates=grammar_rule[target]
        #print(candidates) 
        candidate=random.choice(candidates)
        #print(candidate) 
        return ''.join(generate(grammar_rule,target=c.strip()) for c in candidate.split())
    else :
        return target

In [26]:
def get_generation_by_gram(grammar_str: str,target,stmt_split='=', or_split='|'):
    rules=dict()
    for line in grammar_str.split('\n'):
        if not line:continue
        stmt,expr=line.split(stmt_split)
        rules[stmt.strip()]=expr.split(or_split)
    return generate(rules,target)

In [27]:
get_generation_by_gram(teacher,target='teacher',stmt_split='=')

'体育助教听大学生'

In [40]:
get_generation_by_gram(student,target='student',stmt_split='=')

'我愿意学物理'

TODO: 然后，定义一个函数，generate_n，将generate扩展，使其能够生成n个句子:

In [140]:
def generate_n(num:int,gram_rule:str,target,stmt_split='='):
    i=0
    sentences=[]
    while i<=num:
        sentences.append(get_generation_by_gram(gram_rule,target,stmt_split))
        i+=1
    return sentences

In [141]:
generate_n(5,student,target='student',stmt_split='=')

['我不想听数学', '我不想练物理', '你愿意学数学', '他愿意练数学', '他不想学语文', '我愿意练语文']

#### 2. 使用新数据源完成语言模型的训练

按照我们上文中定义的`prob_2`函数，我们更换一个文本数据源，获得新的Language Model:

1. 下载文本数据集（你可以在以下数据集中任选一个，也可以两个都使用）
    + 可选数据集1，保险行业问询对话集： https://github.com/Computing-Intelligence/insuranceqa-corpus-zh/raw/release/corpus/pool/train.txt.gz
    + 可选数据集2：豆瓣评论数据集：https://github.com/Computing-Intelligence/datasource/raw/master/movie_comments.csv
2. 修改代码，获得新的**2-gram**语言模型
    + 进行文本清洗，获得所有的纯文本
    + 将这些文本进行切词
    + 送入之前定义的语言模型中，判断文本的合理程度

In [80]:
import pandas as pd

trainFile = 'movie_comments.csv'

allData = pd.read_csv(trainFile)

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:
allData.comment.head()

0                                   吴京意淫到了脑残的地步，看了恶心想吐
1    首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...
2    吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...
3                        凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。
4                                                 中二得很
Name: comment, dtype: object

In [85]:
len(allData)

261497

In [86]:
allData.comment[:5]

0                                   吴京意淫到了脑残的地步，看了恶心想吐
1    首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...
2    吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...
3                        凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。
4                                                 中二得很
Name: comment, dtype: object

In [89]:
#进行文本清洗，获得所有的纯文本
import re
def clean_zh_text(text):
    # keep English, digital and Chinese
    comp = re.compile('[^A-Z^a-z^0-9^\u4e00-\u9fa5]')
    return comp.sub('', text)

In [97]:
df_comment=allData['comment'].apply(lambda x:clean_zh_text(str(x)))

In [98]:
df_comment.head()

0                                    吴京意淫到了脑残的地步看了恶心想吐
1    首映礼看的太恐怖了这个电影不讲道理的完全就是吴京在实现他这个小粉红的英雄梦各种装备轮番上场视...
2    吴京的炒作水平不输冯小刚但小刚至少不会用主旋律来炒作吴京让人看了不舒服为了主旋律而主旋律为了...
3                               凭良心说好看到不像战狼1的续集完虐湄公河行动
4                                                 中二得很
Name: comment, dtype: object

In [100]:
#将这些文本进行切词
import jieba

In [102]:
len(df_comment)

261497

In [103]:
def cut(string):
    return list(jieba.cut(string))

In [115]:
TOKENS=[]
for i in range(0,100000):
#     print(df_comment.iloc[i])
    temp = cut(df_comment.iloc[i])
    TOKENS+=temp

In [116]:
len(TOKENS)

1842572

In [117]:
TOKENS[:10]

['吴京', '意淫', '到', '了', '脑残', '的', '地步', '看', '了', '恶心']

In [118]:
# 送入之前定义的语言模型中，判断文本的合理程度
from collections import Counter

In [119]:
%matplotlib inline

In [120]:
words_count = Counter(TOKENS)

In [121]:
words_count.most_common(20)

[('的', 127398),
 ('了', 44535),
 ('是', 29952),
 ('我', 21014),
 ('都', 15430),
 ('看', 14090),
 ('电影', 13545),
 ('也', 13437),
 ('很', 13356),
 ('和', 12586),
 ('在', 12181),
 ('不', 11862),
 ('有', 11600),
 ('就', 11161),
 ('人', 9398),
 ('好', 9283),
 ('啊', 8984),
 ('还', 7740),
 ('你', 7569),
 ('这', 7333)]

In [122]:
_2_gram_words = [
    TOKENS[i] + TOKENS[i+1] for i in range(len(TOKENS)-1)
]

In [123]:
_2_gram_words[:10]

['吴京意淫', '意淫到', '到了', '了脑残', '脑残的', '的地步', '地步看', '看了', '了恶心', '恶心想']

In [124]:
_2_gram_word_counts = Counter(_2_gram_words)

In [125]:
_2_gram_word_counts.most_common(20)

[('的电影', 3274),
 ('看的', 2768),
 ('都是', 2558),
 ('让人', 2226),
 ('看了', 1999),
 ('的是', 1808),
 ('的时候', 1769),
 ('也是', 1760),
 ('的人', 1677),
 ('的故事', 1618),
 ('看完', 1553),
 ('我的', 1386),
 ('让我', 1361),
 ('这部电影', 1162),
 ('的片子', 1157),
 ('这样的', 1084),
 ('了我', 1062),
 ('不知道', 1028),
 ('很好', 1020),
 ('的感觉', 1001)]

In [126]:
def get_1_gram_count(word):
    if word in words_count: return words_count[word]
    else:
        return words_count.most_common()[-1][-1]

In [127]:
def get_2_gram_count(word):
    if word in _2_gram_word_counts: return _2_gram_word_counts[word]
    else:
        return _2_gram_word_counts.most_common()[-1][-1]

In [128]:
def get_gram_count(word, wc):
    if word in wc: return wc[word]
    else:
        return wc.most_common()[-1][-1]

In [129]:
def two_gram_model(sentence):
    # 2-gram langauge model
    tokens = cut(sentence)
    
    probability = 1
    
    for i in range(len(tokens)-1):
        word = tokens[i]
        next_word = tokens[i+1]
        
        _two_gram_c = get_gram_count(word+next_word, _2_gram_word_counts)
        _one_gram_c = get_gram_count(next_word, words_count)
        pro =  _two_gram_c / _one_gram_c
        
        probability *= pro
    
    return probability  

In [130]:
two_gram_model('吴京意淫到了脑残的地步看了恶心想吐')

7.286162102602622e-24

In [131]:
two_gram_model('吴京意淫')

0.01764705882352941

In [132]:
two_gram_model('吴京')

1

#### 3. 获得最优质的的语言

当我们能够生成随机的语言并且能判断之后，我们就可以生成更加合理的语言了。请定义 generate_best 函数，该函数输入一个语法 + 语言模型，能够生成**n**个句子，并能选择一个最合理的句子: 



In [173]:
def generate_best(num:int,gram_rule:str,target,stmt_split='='):
    sentences_list=[]
    sentences=generate_n(num,gram_rule,target,stmt_split)
    for sentence in sentences:
        two_gram_model(sentence)
        sentences_list.append((sentence,two_gram_model(sentence)))
    best_sen=sorted(sentences_list, key=lambda x: x[1], reverse=True)[0]
    return best_sen
        

In [174]:
generate_best(5,student,target='student',stmt_split='=')

('我愿意练语文', 0.004815409309791332)

## 基础理论部分

#### 0. Can you come up out 3 sceneraies which use AI methods? 

自然语言生成 图像识别 推荐系统

#### 1. How do we use Github; Why do we use Jupyter and Pycharm;

github用于上传，共享代码；Jupyter和Pycharm用于编辑python 使用方便

#### 2. What's the Probability Model?

生成的结果是一个概率，预测的结果的可能性

#### 3. Can you came up with some sceneraies at which we could use Probability Model?

评分卡建模 流失率模型

#### 4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

工作量大且会根据实际需要需要频繁修改代码

#### 5. What's the Language Model;

统计语言模型是一个单词序列上的概率分布，对于一个给定长度为m的序列，它可以为整个序列产生一个概率 P(w_1,w_2,…,w_m) 。其实就是想办法找到一个概率分布，它可以表示任意一个句子或序列出现的概率

#### 6. Can you came up with some sceneraies at which we could use Language Model?

目前在自然语言处理相关应用非常广泛，如语音识别(speech recognition) , 机器翻译(machine translation), 词性标注(part-of-speech tagging), 句法分析(parsing)等

#### 7. What's the 1-gram language model;

只统计切分后的单独的一个词在文本中出现的次数

#### 8. What's the disadvantages and advantages of 1-gram language model;

没有后前后出现词的统计，不合逻辑

#### 9. What't the 2-gram models;

n=2 即出现在第 i 位上的词 wi，仅仅于它前一个历史词 wi−1 有关。可以近似的认为，一个词的概率只依赖于它前面的一个词